In [ ]:
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForQuestionAnswering, GPT2LMHeadModel, GPT2Tokenizer
from pyngrok import ngrok
import torch
from flask_cors import CORS # Enable cross-origin requests
import random

torch.cuda.empty_cache()
# Set up a tunnel on the port where Flask runs (default is 5000)
public_url = ngrok.connect(5000)
print("🔗 ngrok tunnel URL:", public_url, flush=True)
# Initialize Flask app
app = Flask(__name__)
CORS(app) # Allow requests from client-side
# Check if a GPU (CUDA) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Using device: {device}", flush=True)
# Model paths
MODEL_PATHS = {
'classification': "/content/drive/MyDrive/dl/biobert_classification",
'qa': "/content/drive/MyDrive/dl/biobert_qa_model",
'gpt2': "/content/drive/MyDrive/dl/fine_tuned_gpt2"
}
# Load models and tokenizers
cls_tokenizer = AutoTokenizer.from_pretrained(MODEL_PATHS['classification'])
cls_model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATHS['classification']).to(device)
qa_tokenizer = AutoTokenizer.from_pretrained(MODEL_PATHS['qa'])
qa_model = AutoModelForQuestionAnswering.from_pretrained(MODEL_PATHS['qa']).to(device)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained(MODEL_PATHS['gpt2'])
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token
gpt2_model = GPT2LMHeadModel.from_pretrained(MODEL_PATHS['gpt2']).to(device)

def generate_patient_profile():
    """
    Generate a comprehensive patient profile with randomized but realistic medical details
    """
    # Predefined lists for randomization
    cancer_types = ['Breast', 'Lung', 'Prostate', 'Colorectal', 'Pancreatic']
    cancer_stages = ['I', 'II', 'III', 'IV']
    treatment_options = [
        {'name': 'Hormone Therapy', 'confidence': 52.0},
        {'name': 'Radiation Therapy', 'confidence': 18.0},
        {'name': 'Surgery', 'confidence': 17.0},
        {'name': 'Chemotherapy', 'confidence': 13.0}
    ]

    # Generate patient profile
    profile = {
        'PATIENT PROFILE': {
            'Age': random.randint(45, 75),
            'Cancer Type': random.choice(cancer_types),
            'Stage': random.choice(cancer_stages),
            'Metastasis': random.choice([True, False]),
            'Tumor Size': round(random.uniform(1.0, 5.0), 1),
            'Previous Treatment': random.choice([True, False]),
            'Comorbidity Score': random.randint(0, 3),
            'Genetic Markers': {
                'Marker 1': random.choice(['Positive', 'Negative']),
                'Marker 2': random.choice(['Positive', 'Negative'])
            },
            'Performance Status': random.randint(0, 2)
        },
        'RECOMMENDED TREATMENT': {},
        'ALTERNATIVE TREATMENT OPTIONS': [],
        'FOLLOW-UP PLAN': {
            'Schedule': 'Every 3 months for the first 2 years',
            'Recommended tests': [
                'Blood work',
                'Imaging',
                'Physical examination'
            ]
        }
    }

    # Select primary treatment
    primary_treatment = max(treatment_options, key=lambda x: x['confidence'])
    profile['RECOMMENDED TREATMENT'] = {
        'Treatment': primary_treatment['name'],
        'Confidence': primary_treatment['confidence']
    }

    # Select alternative treatments (excluding primary)
    alternative_treatments = [
        opt for opt in treatment_options
        if opt['name'] != primary_treatment['name']
    ]
    profile['ALTERNATIVE TREATMENT OPTIONS'] = [
        f"{t['name']} (Confidence: {t['confidence']}%)"
        for t in sorted(alternative_treatments, key=lambda x: x['confidence'], reverse=True)[:3]
    ]

    return profile

@app.route("/get-response", methods=["GET"])
def get_response():
    message = request.args.get('message')
    if not message:
        return jsonify({"error": "Message parameter is missing"}), 400

    # Check if the message is related to patient profile
    if 'profile' in message.lower():
        # Generate and return patient profile
        profile = generate_patient_profile()

        # Format the response to match the desired output style
        formatted_response = "PATIENT PROFILE: " + \
            "-" * 50 + "\n" + \
            "\n".join([f"{k}: {v}" for k, v in profile['PATIENT PROFILE'].items() if k != 'Genetic Markers']) + \
            "\nGenetic Markers:\n" + \
            "\n".join([f"  {k}: {v}" for k, v in profile['PATIENT PROFILE']['Genetic Markers'].items()]) + \
            "\n\nRECOMMENDED TREATMENT: " + \
            "-" * 50 + "\n" + \
            f"Treatment: {profile['RECOMMENDED TREATMENT']['Treatment']}\n" + \
            f"Confidence: {profile['RECOMMENDED TREATMENT']['Confidence']}%\n\n" + \
            "ALTERNATIVE TREATMENT OPTIONS: " + \
            "-" * 50 + "\n" + \
            "\n".join([f"{i+1}. {opt}" for i, opt in enumerate(profile['ALTERNATIVE TREATMENT OPTIONS'])]) + \
            "\n\nFOLLOW-UP PLAN: " + \
            "-" * 50 + "\n" + \
            f"Schedule: {profile['FOLLOW-UP PLAN']['Schedule']}\n" + \
            "Recommended tests:\n" + \
            "\n".join([f"- {test}" for test in profile['FOLLOW-UP PLAN']['Recommended tests']])

        return jsonify({
            'raw_profile': profile,
            'formatted_profile': formatted_response,
            'response': formatted_response
        })

    # Classification
    inputs = cls_tokenizer(message, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = cls_model(**inputs)
        category = torch.argmax(outputs.logits).item()

    # Question Answering - Using a real medical context
    context = """
Cancer is a disease characterized by the uncontrolled growth of abnormal cells in the body.
These cells can invade nearby tissues and spread through the blood and lymph systems.
Common types of cancer include breast cancer, lung cancer, and leukemia.

Cancer begins when genetic changes interfere with the normal cell growth and division process. Cells begin to grow uncontrollably, forming masses called tumors (except in blood cancers). Tumors can be benign (non-cancerous) or malignant (cancerous). Malignant tumors can invade nearby tissues and spread to distant parts of the body.

Cancer types include carcinomas (arising from epithelial cells), sarcomas (developing in connective tissues), leukemias (cancers of blood-forming tissues), lymphomas (cancers of the lymphatic system), myelomas (cancers of plasma cells), and CNS cancers (tumors in the brain or spinal cord).

Cancer staging describes the extent of the cancer: Stage 0 (carcinoma in situ), Stage I (small and contained), Stages II & III (larger with possible spread to surrounding tissues), and Stage IV (metastasized to distant organs).

Risk factors include age, tobacco use, sun exposure, radiation exposure, certain chemicals, chronic inflammation, obesity, alcohol consumption, certain infections, and genetic predisposition.

Diagnosis typically involves physical examination, medical history, imaging tests, laboratory tests, biopsy, genetic testing, and endoscopy procedures.

Treatment options include surgery, radiation therapy, chemotherapy, immunotherapy, targeted therapy, hormone therapy, stem cell transplant, and precision medicine approaches.

Prevention strategies include avoiding tobacco, maintaining a healthy weight, regular physical activity, healthy diet, limited alcohol consumption, sun protection, vaccination against cancer-causing infections, regular screening, and knowing family history.

Survival rates vary widely depending on cancer type, stage at diagnosis, age, overall health, and response to treatment. Five-year survival rates range from over 90% for some early-stage cancers to under 10% for certain advanced cancers.

Recent treatment advances include CAR T-cell therapy, oncolytic virus therapy, CRISPR gene editing, liquid biopsies, AI-assisted diagnostics, novel combination therapies, and personalized cancer vaccines.
"""
    inputs_qa = qa_tokenizer(message, context, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs_qa = qa_model(**inputs_qa)
        answer_start = torch.argmax(outputs_qa.start_logits)
        answer_end = torch.argmax(outputs_qa.end_logits) + 1
        answer = (
            qa_tokenizer.convert_tokens_to_string(
                qa_tokenizer.convert_ids_to_tokens(inputs_qa["input_ids"][0][answer_start:answer_end])
            )
            if answer_end > answer_start
            else "No clear answer found"
        )

    # GPT-2 Refinement
    prompt = f"Rephrase this medical information for a doctor:\n{message}"
    inputs_gpt2 = gpt2_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs_gpt2 = gpt2_model.generate(
        input_ids=inputs_gpt2["input_ids"],
        attention_mask=inputs_gpt2["attention_mask"],
        max_length=100,
        num_return_sequences=1,
        temperature=0.5,
        top_k=50,
        repetition_penalty=1.2,
        pad_token_id=gpt2_tokenizer.eos_token_id
    )
    refined_text = gpt2_tokenizer.decode(outputs_gpt2[0], skip_special_tokens=True).replace(prompt, "").strip()

    # Match response key with client-side expectation
    response_data = {
        "category": category,
        "answer": answer,
        "response": refined_text
    }
    print("✅ Sending Response:", response_data, flush=True)  # Debugging print

    # Return response with correct headers
    return jsonify(response_data), 200

if __name__ == '__main__':
    app.run(port=5000, debug=True, use_reloader=False)

🔗 ngrok tunnel URL: NgrokTunnel: "https://2124-34-91-130-236.ngrok-free.app" -> "http://localhost:5000"
🖥️ Using device: cpu
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


✅ Sending Response: {'category': 2, 'answer': '[CLS]', 'response': "? Cancer can be caused by any number of causes, including genetic mutations or environmental factors. The most common cause of death in humans and animals are cancers that have been identified as early stage (i) carcinogenic to the body's cells; (<1% chance), (>2% chance), <3%, >4%. In addition there may also exist other types such anaerobic bacteria which produce toxins from their environment causing them to"}


INFO:werkzeug:127.0.0.1 - - [29/Mar/2025 09:39:06] "GET /get-response?message=What%20is%20cancer HTTP/1.1" 200 -


✅ Sending Response: {'category': 5, 'answer': '[CLS]', 'response': "It's the most common form of heart disease. The cause and severity of it varies greatly from person to individual, but usually occurs in people with an enlarged or inflamed chest cavity (chest pain). People who have had multiple surgeries are at higher risk of developing bronchitis due either direct exposure to asbestos dust that has been exposed through inhalation into their lungs during surgery—or by inhaling fumes produced when they breathe"}


INFO:werkzeug:127.0.0.1 - - [29/Mar/2025 09:48:40] "GET /get-response?message=What%20is%20lung%20cancer? HTTP/1.1" 200 -


In [1]:
!lsof -i :5000
!kill 19707
!pip install transformers datasets torch matplotlib datasets
!pip install flask torch datasets
!pip install flask-cors
!pip install pyngrok
!pip install flask-ngrok CORS
!ngrok config add-authtoken

/bin/bash: line 1: kill: (19707) - No such process
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
huggingface-cli login

SyntaxError: invalid syntax (<ipython-input-6-f67f8e96d4cc>, line 1)